In [14]:
import pandas as pd
import pulp

#import necessary data
demand_all = pd.read_csv(r'Datasets/demand.csv', sep = ';')
dist_bt_stores = pd.read_csv(r'Datasets/dist_bt_stores.csv', sep = ';')
dist_from_dc = pd.read_csv(r'Datasets/dist_from_dc.csv', sep = ';')
vehicle_restrict_all = pd.read_csv(r'Datasets/vehicle_restrict.csv', sep = ';')
truck_costs_all = pd.read_csv(r'Datasets/truck_costs.csv', sep = ';')

In [15]:
regions = vehicle_restrict_all['Group'].unique()
for r in regions:
    if r == '1':
        truck_costs = truck_costs_all
        vehicle_restrict = vehicle_restrict_all[vehicle_restrict_all['Group'] == r]
        demand = demand_all[demand_all['store'].isin(vehicle_restrict['Code'].unique())]

demand = demand[demand['store'] != 6989] #missing information about store
demand = demand[demand['store'] != 7940] #missing information about store

vehicle_restrict = vehicle_restrict[vehicle_restrict['Code'] != 6989]
vehicle_restrict = vehicle_restrict[vehicle_restrict['Code'] != 7940]

stores = demand['store'].unique().tolist()  #stores 2 to 9000... 205 stores active

food_types = demand['type'].unique().tolist() #FRESH and DRY

days = demand['date'].unique().tolist() # 02 may to 07 may

regions = dist_bt_stores['Group'].tolist() #group 1 to 10

store_region_correspondence = vehicle_restrict[['Code','Group']]

distance_stores = dist_bt_stores['average distance between stores [km]'].tolist()

distance_dc = dist_from_dc['distance to group [km]*'].tolist() #distance of group 1 is 98..

vehicle_size_limit = vehicle_restrict[vehicle_restrict['Code'].isin(stores)]['Vehicle size limit'].tolist() #R13 for store 2..

max_loading_docks = 10

vehicle_types = truck_costs['vehicle'].unique().tolist() #R13...

vehicle_pallet_capacity = truck_costs['pallet capacity'].tolist()

fixed_costs_per_day = truck_costs['€/day'].tolist()

per_hour_costs = truck_costs['€/h'].tolist()

per_km_costs = truck_costs['€/km'].tolist()

average_fixed_loading_time_store = 15/60 #hours

average_variable_loading_time_store = 80/(60*60) #hours

average_fixed_unloading_time_store = 15/60 #hours

average_variable_unloading_time_store = 120/(60*60)

average_truck_speed_from_dc2store_dock = 75 #km/h

average_truck_speed_bt_stores_or_dock2store = 30

pallet_size = 1 #sqm

dock_daily_cost = 10/30 #€/sqm/day

operator_cost = 10 #€

min_dock_area = 300 #sqm

average_fixed_unloading_time_dock = 15/60

average_variable_unloading_time_dock = 60/(60*60)

average_fixed_loading_time_dock = 15/60

average_variable_loading_time_dock = 15/(60*60)

average_sorting_time_dock = 120/(60*60)

dock_cost = [4, 2] #cost/sqm of cold; dry

number_vehicles_type1 = 0 ## adjust accordingly to region needs

number_vehicles_type2 = 0

number_vehicles_type3 = 10

docks = list(range(1))
docks = [x + 1 for x in docks] #maximum of one dock per region 
vehicle_list = []

# Appending vehicle types to the list based on the specified counts
for i in range(number_vehicles_type1):
    vehicle_list.append(vehicle_types[0])
for i in range(number_vehicles_type2):
    vehicle_list.append(vehicle_types[1])
for i in range(number_vehicles_type3):
    vehicle_list.append(vehicle_types[2])
vehicle_type_correspondence = pd.DataFrame(vehicle_list, columns=['Type'])
vehicle_type_correspondence['Index'] = range(0, len(vehicle_type_correspondence))
store_region_correspondence['Index'] = range(0, len(store_region_correspondence))

#create a demand array per day, store and tyoe (3 dimensions)
demand_array = []
for i in demand['date'].unique():
    lst1 = []
    for j in demand['store'].unique():
        lst2 = []
        for k in demand['type'].unique():
            try:
                demand_value = demand.loc[(demand['date'] == i) & (demand['type'] == k) & (demand['store'] == j)]['pallets'].iloc[0]
                demand_value = int(demand_value)
                lst2.append(demand_value)
            except:
                lst2.append(0)
        lst1.append(lst2)
    demand_array.append(lst1)

C:\Users\vasco\AppData\Local\Temp\ipykernel_1520\3816942601.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_region_correspondence['Index'] = range(0, len(store_region_correspondence))


In [16]:
#variable if makes trajectory i-->j in vehicle k in day d
x = {(d, k, i, j): pulp.LpVariable(f'x{d}_{k}_{i}_{j}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for i in range(len(stores))
        for j in range(len(stores))}

#variable of number_of_pallets in trajectory i-->j in vehicle k in day d
number_pallets_bt_stores = {(d, k, i, j): pulp.LpVariable(f'number_pallets_bt_stores{d}_{k}_{i}_{j}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for i in range(len(stores))
        for j in range(len(stores))}

#variable if unloading happens in store j
z = {(d, k, j): pulp.LpVariable(f'z{d}_{k}_{j}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for j in range(len(stores))}

#variable if vehicle goes from the main warehouse to place j
u = {(d, k, j): pulp.LpVariable(f'u{d}_{k}_{j}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for j in range(len(stores))}

#variable of number_of_pallets from warehouse to store i
number_pallets_from_warehouse_to_stores = {(d, k, j): pulp.LpVariable(f'number_pallets_from_warehouse_to_stores{d}_{k}_{j}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for j in range(len(stores))}

#variable of number_of_pallets from warehouse to dock i
number_pallets_from_warehouse_to_dock = {(d, k, s): pulp.LpVariable(f'number_pallets_from_warehouse_to_dock{d}_{k}_{s}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))}

#variable of number_of_pallets from warehouse to store i
number_pallets_from_dock_to_store = {(d, k, s, j): pulp.LpVariable(f'number_pallets_from_dock_to_store{d}_{k}_{s}_{j}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))
        for j in range(len(stores))}

#variable if vehicle goes from the place i to main warehouse 
v = {(d, k, i): pulp.LpVariable(f'v{d}_{k}_{i}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for i in range(len(stores))}

#variable if vehicle goes from the dock s to store j
w = {(d, k, s, j): pulp.LpVariable(f'w{d}_{k}_{s}_{j}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))
        for j in range(len(stores))}

#Assume no dock to warehouse, needs to go to a store

#variable if vehicle goes from the main warehouse to dock s
l = {(d, k, s): pulp.LpVariable(f'l{d}_{k}_{s}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))}

#variable if loading happens in dock
p = {(d, k, s): pulp.LpVariable(f'p{d}_{k}_{s}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))}

#variable if unloading happens in dock
q = {(d, k, s): pulp.LpVariable(f'q{d}_{k}_{s}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))}

#variable of number of pallets in dock s in day d
number_pallets_dock = {(d, s): pulp.LpVariable(f'number_pallets_dock{d}_{s}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for s in range(len(docks))}

#Area of dock
dock_area = {(s): pulp.LpVariable(f'dock_area_{s}', cat = 'Integer')
        for s in range(len(docks))}

#variable of number_of_pallets in trajectory i-->j in vehicle k in day d by type
number_pallets_bt_stores_types = {(d, k, i, j ,t): pulp.LpVariable(f'number_pallets_bt_stores_types{d}_{k}_{i}_{j}_{t}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for i in range(len(stores))
        for j in range(len(stores))
        for t in range(len(food_types))}

#variable of number_of_pallets warehouse to store by type
number_pallets_from_warehouse_to_stores_type = {(d, k, j, t): pulp.LpVariable(f'number_pallets_from_warehouse_to_stores_types{d}_{k}_{j}_{t}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for j in range(len(stores))
        for t in range(len(food_types))}

#variable of number_of_pallets from warehouse to dock i by type
number_pallets_from_warehouse_to_dock_types = {(d, k, s, t): pulp.LpVariable(f'number_pallets_from_warehouse_to_dock_types{d}_{k}_{s}_{t}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))
        for t in range(len(food_types))}

#variable of number_of_pallets from warehouse to store i
number_pallets_from_dock_to_store_types = {(d, k, s, j, t): pulp.LpVariable(f'number_pallets_from_dock_to_store_types{d}_{k}_{s}_{j}_{t}', cat= 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))
        for j in range(len(stores))
        for t in range(len(food_types))}

#variable of number of pallets in dock per day and type
number_pallets_dock_types = {(d, s, t): pulp.LpVariable(f'number_pallets_dock_types{d}_{s}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for s in range(len(docks))
        for t in range(len(food_types))}

#variable of number of pallets that went into dock per day and type
number_of_pallets_into_dock_types = {(d, s, t): pulp.LpVariable(f'number_of_pallets_into_dock_types{d}_{s}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for s in range(len(docks))
        for t in range(len(food_types))}

#variable of number of pallets that went into dock per day and type 
number_of_pallets_out_dock_types = {(d, s, t): pulp.LpVariable(f'number_of_pallets_out_dock_types{d}_{s}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for s in range(len(docks))
        for t in range(len(food_types))}

#variable of number of pallets that went out dock per day and type and vehicle
number_of_pallets_out_dock_types_per_vehicle= {(d, k, s, t): pulp.LpVariable(f'number_of_pallets_out_dock_types_per_vehicle{d}_{k}_{s}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))
        for t in range(len(food_types))}

#variable of number of pallets that went into dock per day and type and vehicle
number_of_pallets_into_dock_types_per_vehicle= {(d, k, s, t): pulp.LpVariable(f'number_of_pallets_into_dock_types_per_vehicle{d}_{k}_{s}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for s in range(len(docks))
        for t in range(len(food_types))}

#variable of number of pallets that went into store per day and type
number_of_pallets_into_store_types = {(d, j, t): pulp.LpVariable(f'number_of_pallets_into_store_types{d}_{j}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for j in range(len(stores))
        for t in range(len(food_types))}

#variable of number of pallets that went into store per day and type and vehicle
number_of_pallets_into_store_types_per_vehicle = {(d, k, j, t): pulp.LpVariable(f'number_of_pallets_into_store_types_per_vehicle{d}_{k}_{j}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for j in range(len(stores))
        for t in range(len(food_types))}

#variable of number of pallets from store to warehouse per type
number_pallets_store_to_warehouse_types = {(d, i, t): pulp.LpVariable(f'number_pallets_store_to_warehouse_types{d}_{i}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for i in range(len(stores))
        for t in range(len(food_types))}

#variable of number of pallets from store to warehouse per type and vehicle
number_pallets_store_to_warehouse_types_per_vehicle = {(d, k, i, t): pulp.LpVariable(f'number_pallets_store_to_warehouse_types_per_vehicle{d}_{k}_{i}_{t}', cat = 'Integer', lowBound=0)
        for d in range(len(days))
        for k in range(max_loading_docks)
        for i in range(len(stores))
        for t in range(len(food_types))}

#variable if dock exists in region
dock_exists = {(s): pulp.LpVariable(f'dock_exists{s}', cat=pulp.LpBinary)
        for s in range(len(docks))}

#variable if truck is used in day
truck_used = {(d,k): pulp.LpVariable(f'truck_used{d}_{k}', cat=pulp.LpBinary)
        for d in range(len(days))
        for k in range(max_loading_docks)}

In [17]:
prob = pulp.LpProblem("VRP", pulp.LpMinimize)

#Cost between stores
c1 = pulp.lpSum(x[(d, k, i, j)] *  (  \
                                        + distance_stores[int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0])-1] * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/km'].iloc[0]\
                                        + distance_stores[int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0])-1]/(average_truck_speed_bt_stores_or_dock2store) * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0])\
                    for d in range(len(days)) for k in range(max_loading_docks) for i in range(len(stores)) for j in range(len(stores)))

#Cost from warehouse to store j
c2 = pulp.lpSum(u[(d, k, j)] *  (  \
                                        + distance_dc[int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0])-1] * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/km'].iloc[0]\
                                        + distance_dc[int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0])-1]/(average_truck_speed_from_dc2store_dock) * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0])\
                    for d in range(len(days)) for k in range(max_loading_docks) for j in range(len(stores)))

#Cost from store i to warehouse
c3 = pulp.lpSum(v[(d, k, i)] *  (  \
                                        + distance_dc[int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0])-1] * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/km'].iloc[0]\
                                        + distance_dc[int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0])-1]/(average_truck_speed_from_dc2store_dock) * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0])\
                    for d in range(len(days)) for k in range(max_loading_docks) for i in range(len(stores)))

#Cost from dock to store j
c4 = pulp.lpSum(w[(d, k, s, j)] *  ( \
                                        + distance_stores[int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0])-1] * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/km'].iloc[0]\
                                        + distance_stores[int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0])-1]/(average_truck_speed_bt_stores_or_dock2store) * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0])\
                    for d in range(len(days)) for k in range(max_loading_docks) for s in range(len(docks)) for j in range(len(stores)))

#Cost from main warehouse to dock
c5 = pulp.lpSum(l[(d, k, s)] *  (  \
                                        + distance_dc[docks[s]-1] * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/km'].iloc[0]\
                                        + distance_dc[docks[s]-1]/(average_truck_speed_from_dc2store_dock) * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0])\
                    for d in range(len(days)) for k in range(max_loading_docks) for s in range(len(docks)))

#Cost of area of all docks
c6 = pulp.lpSum(dock_daily_cost * dock_exists[w] * min_dock_area + dock_area[w] for w in range(len(docks)))

#Cost of storage of pallets in all docks
c7 = pulp.lpSum(dock_cost[t] * pallet_size * number_pallets_dock_types[(d,w,t)] for d in range(len(days)) for w in range(len(docks)) for t in range(len(food_types)))

#Cost of unloading pallets to dock
c8 = pulp.lpSum(operator_cost * (average_fixed_unloading_time_dock + (average_variable_unloading_time_dock + average_sorting_time_dock) * number_of_pallets_into_dock_types[(d,w,t)]) for d in range(len(days)) for w in range(len(docks)) for t in range(len(food_types)))

#Cost of loading pallets from dock
c9 = pulp.lpSum(operator_cost * (average_fixed_loading_time_dock + (average_variable_loading_time_dock) * number_of_pallets_out_dock_types[(d,w,t)]) for d in range(len(days)) for w in range(len(docks)) for t in range(len(food_types)))

#Cost per hour for unloading in store
c10 = pulp.lpSum((average_variable_unloading_time_store * number_of_pallets_into_store_types_per_vehicle[(d,k,j,t)] * truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0] + truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/h'].iloc[0] * average_fixed_unloading_time_store) for d in range(len(days)) for k in range(max_loading_docks) for j in range(len(stores)) for t in range(len(food_types)))
    
#Cost per day if truck is used
c11 = pulp.lpSum(truck_used[(d,k)]* truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['€/day'].iloc[0] for d in range(len(days)) for k in range(max_loading_docks))
    
prob += c1 + c2 +c3 +c4 +c5 +c6 +c7 + c8 +c9 +c10 +c11

In [18]:
#Can´t be pallets out of dock in day one
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for t in range(len(food_types)):
                if d==0:
                    prob += number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] == 0
                    prob += number_pallets_dock_types[(d,s,t)] == 0
                    prob += number_pallets_dock[(d,s)] == 0

#The number of pallets times their size does not exceed the area of the dock
for d in range(len(days)):
    for s in range(len(docks)):
        prob += number_pallets_dock[(d,s)] * pallet_size <= dock_area[s] 

#If dock doesn't exist, its area is 0
for s in range(len(docks)):
    prob += dock_area[s] <= 100000 * dock_exists[s]

#The area of the dock should be higher than the minimum
for s in range(len(docks)):
    prob += dock_area[s] >= min_dock_area * dock_exists[s]

#A movement from a dock can only happen if the dock exists
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                prob += w[(d,k,s,j)] <= dock_exists[s]

#A movement from dock to warehouse can only happen if dock exists
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += l[(d,k,s)] <= dock_exists[s]
            prob += p[(d,k,s)] <= dock_exists[s]
            prob += q[(d,k,s)] <= dock_exists[s]

#The number of pallets in a dock can be bigger than 0 if the docks exists
for d in range(len(days)):
    for s in range(len(docks)):
        prob += number_pallets_dock[(d,s)] <= dock_exists[s] * 100000
    
#the transportation of pallets only happens if the dock exists
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += number_pallets_from_warehouse_to_dock[(d,k,s)] <= dock_exists[s] * 100000
            for t in range(len(food_types)):
                prob += number_pallets_from_warehouse_to_dock_types[(d,k,s,t)] <= dock_exists[s] * 100000
    
#The transportation per type can happen if dock exists
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                prob += number_pallets_from_dock_to_store[(d,k,s,j)] <= dock_exists[s] * 100000
                for t in range(len(food_types)):
                    prob += number_pallets_from_dock_to_store_types[(d,k,s,j,t)] <= dock_exists[s] * 100000

#The transportation of pallets in dock per type, into dock and out dock can only happen if the dock exists
for d in range(len(days)):
    for s in range(len(docks)):
        for t in range(len(food_types)):
            prob += number_pallets_dock_types[(d,s,t)] <= dock_exists[s] * 100000
            prob += number_of_pallets_into_dock_types[(d,s,t)] <= dock_exists[s] * 100000
            prob += number_of_pallets_out_dock_types[(d,s,t)] <= dock_exists[s] * 100000

#In out can happen if dock exists
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for t in range(len(food_types)):
                prob += number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] <= dock_exists[s] * 100000
                prob += number_of_pallets_into_dock_types_per_vehicle[(d,k,s,t)] <= dock_exists[s] * 100000

In [19]:
# The maximum duration for a go and return trip including loading and unloading is 10h.
for d in range(len(days)):
    for k in range(max_loading_docks):
                
                #time spent from store i to store j
                constraint_expr1 = pulp.lpSum(x[(d, k, i, j)] * distance_stores[int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0])-1]/(average_truck_speed_bt_stores_or_dock2store) for i in range(len(stores)) for j in range(len(stores)))

                #time spent from unloading in store j 
                constraint_expr1 += pulp.lpSum((average_variable_unloading_time_store * number_of_pallets_into_store_types_per_vehicle[(d,k,j,t)] + average_fixed_unloading_time_store * z[(d,k,j)]) for j in range(len(stores)) for t in range(len(food_types)))
                
                #time spent loading in dock
                constraint_expr1 += pulp.lpSum((average_variable_loading_time_dock * number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] + average_fixed_loading_time_dock * p[(d,k,s)]) for s in range(len(docks)) for t in range(len(food_types)))
                
                #time spent unloading in dock
                constraint_expr1 += pulp.lpSum((average_variable_unloading_time_dock * number_of_pallets_into_dock_types_per_vehicle[(d,k,s,t)] + average_fixed_unloading_time_dock * q[(d,k,s)]) for s in range(len(docks)) for t in range(len(food_types)))
                
                #time spent from warehouse to store j
                constraint_expr1 += pulp.lpSum(u[(d, k, j)] * distance_dc[int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0])-1]/(average_truck_speed_from_dc2store_dock) for j in range(len(stores)))
                
                #time spent from store i to warehouse
                constraint_expr1 += pulp.lpSum(v[(d, k, i)] * distance_dc[int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0])-1]/(average_truck_speed_from_dc2store_dock) for i in range(len(stores)))
                
                #time spent from dock to store j
                constraint_expr1 += pulp.lpSum(w[(d, k, s, j)] * distance_stores[int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0])-1]/(average_truck_speed_bt_stores_or_dock2store) for s in range(len(docks)) for j in range(len(stores)))
                
                #time spent from main warehouse to dock
                constraint_expr1 += pulp.lpSum(l[(d, k, s)] * distance_dc[docks[s]-1]/(average_truck_speed_from_dc2store_dock) for s in range(len(docks)))
                
                prob += constraint_expr1 <= 10

In [20]:
#A vehicle can only go to one place if it obbeys to size limits
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for j in range(len(stores)):
                if vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0] != vehicle_size_limit[j]:
                    prob += x[(d,k,i,j)] == 0
                        
#A vehicle can only go to one place if it obbeys to size limits
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                if vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0] != vehicle_size_limit[j]:
                    prob += w[(d,k,s,j)] == 0

#Number of pallets into store per vehicle < capacity
for d in range(len(days)):
    for k in range(max_loading_docks):
        cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
        for i in range(len(stores)):
            prob += number_of_pallets_into_store_types[(d,j,t)] <= cap
            for t in range(len(food_types)):
                prob += number_of_pallets_into_store_types_per_vehicle[(d,k,i,t)] <= cap

#Number of pallets into dock per vehicle < capacity
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
                cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
                prob += pulp.lpSum(number_of_pallets_into_dock_types_per_vehicle[(d,k,s,t)] for t in range(len(food_types))) <= cap 

#Number of pallets into dock per vehicle < capacity
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for t in range(len(food_types)):
                cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
                prob += pulp.lpSum(number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] for t in range(len(food_types))) <= cap 

#One vehicle in any day cannot exceed the capacity when leaving the dock to the store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
                prob += number_pallets_from_dock_to_store[(d,k,s,j)] <= cap

#What leaves the main warehouse is smaller than the vehicles capacity
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
            cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
            prob += number_pallets_from_warehouse_to_stores[(d,k,j)] <= cap
        for s in range(len(docks)):
            cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
            prob += number_pallets_from_warehouse_to_dock[(d,k,s)] <= cap

#One vehicle in any day cannot exceed the capacity between stores
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for j in range(len(stores)):
                cap = int(truck_costs[truck_costs['vehicle'] == vehicle_type_correspondence[vehicle_type_correspondence['Index'] == k]['Type'].iloc[0]]['pallet capacity'].iloc[0])
                prob += number_pallets_bt_stores[(d,k,i,j)] <= cap

#if truck used
for d in range(len(days)):
    for k in range(max_loading_docks):
        prob += truck_used[(d,k)] * 100000 >= pulp.lpSum(v[(d,k,i)] for i in range(len(stores))) + pulp.lpSum(x[(d,k,i,j)] for i in range(len(stores)) for j in range(len(stores)) ) + pulp.lpSum(l[d,k,s] for s in range(len(docks))) + pulp.lpSum(u[(d,k,j)] for j in range(len(stores))) + pulp.lpSum(w[(d,k,s,j)] for s in range(len(docks)) for j in range(len(stores)))

#Number of pallets out < what is in dock
for d in range(len(days)):
        for k in range(max_loading_docks):
            for s in range(len(docks)):
                for t in range(len(food_types)):
                    prob += number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] <= number_pallets_dock_types[(d,s,t)]

In [21]:
#What goes into a specific store in each day by all vehicles from the different places must be equal to the demand of that day
for d in range(len(days)):
    for j in range(len(stores)):
        for t in range(len(food_types)):
            prob += number_of_pallets_into_store_types[(d,j,t)] >= demand_array[d][j][t]

In [22]:
#the number of total pallets equals the number of pallets of all types bt stores
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for j in range(len(stores)):
                prob += number_pallets_bt_stores[(d,k,i,j)] == pulp.lpSum(number_pallets_bt_stores_types[(d,k,i,j,t)] for t in range(len(food_types)))

#The sum of number of pallets per type is equal to the total
for d in range(len(days)):
    for s in range(len(docks)):
        prob += number_pallets_dock[(d,s)] == pulp.lpSum(number_pallets_dock_types[(d,s,t)] for t in range(len(food_types)))

#the number of total pallets equals the number of pallets of all types from dock to store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                prob += number_pallets_from_dock_to_store[(d,k,s,j)] == pulp.lpSum(number_pallets_from_dock_to_store_types[(d,k,s,j,t)] for t in range(len(food_types)))

#the number of total pallets equals the number of pallets of all types from warehouse to stores
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
            prob += number_pallets_from_warehouse_to_stores[(d,k,j)] == pulp.lpSum(number_pallets_from_warehouse_to_stores_type[(d,k,j,t)] for t in range(len(food_types)))

#the number of total pallets equals the number of pallets of all types from warehouse to dock
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += number_pallets_from_warehouse_to_dock[(d,k,s)] == pulp.lpSum(number_pallets_from_warehouse_to_dock_types[(d,k,s,t)] for t in range(len(food_types)))

#Number of pallets into store = sum of all pallets per vehicle
for d in range(len(days)):
    for i in range(len(stores)):
        for t in range(len(food_types)):
            prob += number_of_pallets_into_store_types[(d,i,t)] == pulp.lpSum(number_of_pallets_into_store_types_per_vehicle[(d,k,i,t)] for k in range(max_loading_docks))

#Number of pallets into/out dock = sum of all pallets per vehicle
for d in range(len(days)):
    for s in range(len(docks)):
        for t in range(len(food_types)):
            prob += number_of_pallets_into_dock_types[(d,s,t)] == pulp.lpSum(number_of_pallets_into_dock_types_per_vehicle[(d,k,s,t)] for k in range(max_loading_docks))
            prob += number_of_pallets_out_dock_types[(d,s,t)] == pulp.lpSum(number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] for k in range(max_loading_docks))

#Number of pallets from store to warehouse = sum of all pallets per vehicle   
for d in range(len(days)):
    for i in range(len(stores)):
        for t in range(len(food_types)):
            prob += number_pallets_store_to_warehouse_types[(d,i,t)] == pulp.lpSum(number_pallets_store_to_warehouse_types_per_vehicle[(d,k,i,t)] for k in range(max_loading_docks))

#Number of pallets from store to warehouse should be 0 --> no waste of transportation
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for t in range(len(food_types)):
                prob += number_pallets_store_to_warehouse_types_per_vehicle[(d,k,i,t)] == 0

In [23]:
#The number of pallets transported from dock to store is majorated by if that trajectory happens
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                prob += number_pallets_from_dock_to_store[(d,k,s,j)] <= 100000 * w[(d,k,s,j)]

#The number of pallets from warehouse to somewhere is majorated by if that trajectory happens
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
            prob += number_pallets_from_warehouse_to_stores[(d,k,j)] <= 100000 * (u[(d, k, j)])

#The number of pallets from warehouse to dock depends on if that trajectory occurs
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += number_pallets_from_warehouse_to_dock[(d,k,s)] <= 100000 * (l[(d,k,s)])

#The number of pallets transported between two stores is majorated by if that trajectory happens
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for j in range(len(stores)):
                prob+= number_pallets_bt_stores[(d,k,i,j)] <= 100000 * x[(d,k,i,j)]

#The number of pallets from store to warehouse is majorated by if that trajectory happens
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for t in range(len(food_types)):
                prob += number_pallets_store_to_warehouse_types_per_vehicle[(d,k,i,t)] <= 100000 * v[(d,k,i)]

#The number of pallets into store <= number between stores
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
                for t in range(len(food_types)):
                        prob += number_of_pallets_into_store_types_per_vehicle[(d,k,j,t)] <= pulp.lpSum(number_pallets_bt_stores_types[(d,k,i,j,t)] for i in range(len(stores))) + pulp.lpSum(number_pallets_from_dock_to_store_types[(d,k,s,j,t)] for s in range(len(docks))) + number_pallets_from_warehouse_to_stores_type[(d,k,j,t)]

#loading happens in dock if there are pallets out the dock
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += pulp.lpSum(number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] for t in range(len(food_types))) <= p[(d,k,s)] * 100000

#unloading happens in dock if there are pallets into dock
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += pulp.lpSum(number_of_pallets_into_dock_types_per_vehicle[(d,k,s,t)] for t in range(len(food_types))) <= q[(d,k,s)] * 100000
            
#unloading happens in store if there are pallets into store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
            prob += pulp.lpSum(number_of_pallets_into_store_types_per_vehicle[(d,k,j,t)] for t in range(len(food_types))) <= z[(d,k,j)] * 100000

In [24]:
#pallet balance in dock
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for t in range(len(food_types)):
                prob += pulp.lpSum(number_pallets_from_dock_to_store_types[(d,k,s,j,t)] for j in range(len(stores))) == number_of_pallets_out_dock_types_per_vehicle[(d,k,s,t)] + number_pallets_from_warehouse_to_dock_types[(d,k,s,t)] - number_of_pallets_into_dock_types_per_vehicle[(d,k,s,t)]

#pallet balance in store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
            for t in range(len(food_types)):
                prob += pulp.lpSum(number_pallets_bt_stores_types[(d,k,j,m,t)] for m in range(len(stores))) + number_pallets_store_to_warehouse_types_per_vehicle[(d,k,j,t)] == pulp.lpSum(number_pallets_bt_stores_types[(d,k,i,j,t)] for i in range(len(stores))) + number_pallets_from_warehouse_to_stores_type[(d,k,j,t)] + pulp.lpSum(number_pallets_from_dock_to_store_types[(d,k,s,j,t)] for s in range(len(docks))) - number_of_pallets_into_store_types_per_vehicle[(d,k,j,t)]

In [25]:
#Updating pallet number
for d in range(len(days)):
    for s in range(len(docks)):
        for t in range(len(food_types)):
           if d>0:
              prob += number_pallets_dock_types[(d,s,t)] == number_pallets_dock_types[(d-1,s,t)] + number_of_pallets_into_dock_types[(d-1,s,t)] - number_of_pallets_out_dock_types[(d-1,s,t)]

In [26]:
#if went to dock, needs to go to at least one store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += l[(d,k,s)] == pulp.lpSum(w[(d,k,s,j)] for j in range(len(stores)))

#if left, then returns
for d in range(len(days)):
    for k in range(max_loading_docks):
        prob += pulp.lpSum(v[(d,k,i)] for i in range(len(stores)))  == pulp.lpSum(l[(d,k,s)] for s in range(len(docks))) + pulp.lpSum(u[(d,k,j)] for j in range(len(stores)))

#In one day for any vehicle, there can only be one movement to the warehouse
for d in range(len(days)):
    for k in range(max_loading_docks):
        prob += pulp.lpSum(v[(d,k,i)] for i in range(len(stores))) <= 1

#In any day any vehicle can only go from the dock s to one store max
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            prob += pulp.lpSum(w[(d,k,s,j)] for j in range(len(stores))) <= 1

#One vehicle in any day can only go to a maximum of one store from a specific store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            prob += pulp.lpSum(x[(d,k,i,j)] for j in range(len(stores))) <= 1

#if a vehicle leaves the warehouse to store j, then there needs to be a trajectory from j somewhere
for d in range(len(days)):
    for k in range(max_loading_docks):
        for j in range(len(stores)):
            prob += u[(d,k,j)] + pulp.lpSum(w[(d,k,s,j)] for s in range(len(docks))) +pulp.lpSum(x[(d,k,i,j)] for i in range(len(stores))) == pulp.lpSum(x[(d,k,j,m)] for m in range(len(stores))) + v[(d,k,j)]
            prob += u[(d,k,j)] + pulp.lpSum(w[(d,k,s,j)] for s in range(len(docks))) +pulp.lpSum(x[(d,k,i,j)] for i in range(len(stores))) <= 1
            prob += pulp.lpSum(x[(d,k,j,m)] for m in range(len(stores))) + v[(d,k,j)] <= 1

#A vehicle can only navigate in a certain region and not between two different regions
for d in range(len(days)):
    for k in range(max_loading_docks):
        for i in range(len(stores)):
            for j in range(len(stores)):
                if int(store_region_correspondence[store_region_correspondence['Index'] == i]['Group'].iloc[0]) != int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0]):
                    prob += x[(d,k,i,j)] == 0

#A vehicle can only navigate in a certain region and not between two different regions to store in a dock from dock to store
for d in range(len(days)):
    for k in range(max_loading_docks):
        for s in range(len(docks)):
            for j in range(len(stores)):
                if int(store_region_correspondence[store_region_correspondence['Index'] == j]['Group'].iloc[0]) != docks[s]:
                    prob += w[(d,k,s,j)] == 0

# In any day in any vehicle the vehicle only leaves the warehouse once, either to the dock or to the store
for d in range(len(days)):
    for k in range(max_loading_docks):
        constraint_expr = pulp.lpSum(u[(d,k,j)] for j in range(len(stores)))
        constraint_expr += pulp.lpSum(l[(d, k, s)] for s in range(len(docks)))
        prob += constraint_expr <= 1

In [27]:
print('Solving....')
time_limit_in_seconds = 120

prob.solve(pulp.PULP_CBC_CMD(msg=1, maxSeconds=time_limit_in_seconds))
print('Done')
print(pulp.value(prob.objective))
print(pulp.LpStatus[prob.status])

Solving....


C:\Users\vasco\AppData\Roaming\Python\Python311\site-packages\pulp\apis\coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Done
15394.320000000022
Optimal


In [28]:
for var in prob.variables():
    if var.varValue != 0:
        print(f"{var.name}: {var.varValue}" )
    else: 
        pass

number_of_pallets_into_store_types0_0_0: 21.0
number_of_pallets_into_store_types0_0_1: 106.0
number_of_pallets_into_store_types0_1_0: 12.0
number_of_pallets_into_store_types0_1_1: 30.0
number_of_pallets_into_store_types0_2_0: 11.0
number_of_pallets_into_store_types0_2_1: 21.0
number_of_pallets_into_store_types0_3_0: 10.0
number_of_pallets_into_store_types0_3_1: 17.0
number_of_pallets_into_store_types0_4_0: 11.0
number_of_pallets_into_store_types0_4_1: 22.0
number_of_pallets_into_store_types0_5_0: 13.0
number_of_pallets_into_store_types0_5_1: 20.0
number_of_pallets_into_store_types1_0_0: 25.0
number_of_pallets_into_store_types1_0_1: 69.0
number_of_pallets_into_store_types1_1_0: 14.0
number_of_pallets_into_store_types1_1_1: 26.0
number_of_pallets_into_store_types1_2_0: 11.0
number_of_pallets_into_store_types1_2_1: 22.0
number_of_pallets_into_store_types1_3_0: 9.0
number_of_pallets_into_store_types1_3_1: 20.0
number_of_pallets_into_store_types1_4_0: 11.0
number_of_pallets_into_store_types